# 환경 변수 로딩

## 와인 reviewdata indexing
-vectorDB : pinecone

In [11]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
LLAMA_EMBEDDING_MODEL = os.getenv("LLAMA_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = 'wine-review2'
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

## doc loader

In [4]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:3]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

In [3]:

len(docs)

129971

In [19]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

NameError: name 'chunks' is not defined

In [23]:
# split하기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [8]:
len(chunks)

129982

In [9]:

chunks[2]

Document(metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 2}, page_content=': 2\ncountry: US\ndescription: Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.\ndesignation: \npoints: 87\nprice: 14.0\nprovince: Oregon\nregion_1: Willamette Valley\nregion_2: Willamette Valley\ntaster_name: Paul Gregutt\ntaster_twitter_handle: @paulgwine\ntitle: Rainstorm 2013 Pinot Gris (Willamette Valley)\nvariety: Pinot Gris\nwinery: Rainstorm')

In [33]:
from pinecone import Pinecone

pc = Pinecone(api_key= PINECONE_API_KEY, environment="us-east1-gcp")

index_name = "llama-text-embed-v2"

index = pc.Index(name=index_name, host="https://wine-review2-znzg6hs.svc.aped-4627-b74a.pinecone.io")

In [20]:
vector_store = PineconeVectorStore(pinecone_index=index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [27]:
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex

# 1. LangChain Document를 LlamaIndex TextNode로 변환
nodes = []
for doc in chunks:
    node = TextNode(
        text=doc.page_content,    # LangChain의 텍스트 내용
        metadata=doc.metadata     # LangChain의 메타데이터(출처, 페이지 등) 유지
    )
    nodes.append(node)

print(f"총 {len(nodes)}개의 노드로 변환되었습니다.")

총 129982개의 노드로 변환되었습니다.


In [34]:
# 2. 배치 단위로 Pinecone에 Upsert 진행
BATCH_SIZE = 500 

for i in range(0, len(nodes), BATCH_SIZE):
    batch_nodes = nodes[i : i + BATCH_SIZE]
    
    if i == 0:
        # 첫 번째 배치로 인덱스 초기화 (Pinecone 연결 및 임베딩 시작)
        index = VectorStoreIndex(
            nodes=batch_nodes, 
            storage_context=storage_context,
            show_progress=True
        )
        print(f"첫 번째 배치({len(batch_nodes)}개) 완료 및 인덱스 생성됨.")
    else:
        # 이후 배치들은 기존 인덱스에 추가
        index.insert_nodes(batch_nodes)
        print(f"{i}번째부터 {i + len(batch_nodes)}번째 노드까지 추가 완료.")

print("--- 모든 데이터가 Pinecone에 저장되었습니다! ---")

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/500 [00:00<?, ?it/s]

첫 번째 배치(500개) 완료 및 인덱스 생성됨.


Upserted vectors:   0%|          | 0/500 [00:00<?, ?it/s]

500번째부터 1000번째 노드까지 추가 완료.


...

55500번째부터 56000번째 노드까지 추가 완료.


Upserted vectors:   0%|          | 0/500 [00:00<?, ?it/s]

56000번째부터 56500번째 노드까지 추가 완료.


Upserted vectors:   0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

In [31]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Admin\miniconda3\envs\lc_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\AppData\Local\llama_index\llama_index\Cache\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]